In [1]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelBinarizer
print("Tensorflow version: ", tf.__version__)

Tensorflow version:  2.7.0


In [2]:
# unpickle all variables
with open('processed_dataset.pickle' , 'rb') as f:
    dataset = pickle.load(f)
    diag = pickle.load(f)
    localization = pickle.load(f)

In [3]:
mapping_dx = {
    'nevus': 0,
    'melanoma': 1,
    'keratosis-like': 0,
    'basal cell carcinoma': 0,
    'vascular lesion': 0,
    'dermatofibroma': 0,
    'lentigo': 0,
}
mapping_localization = {
    'lower extremity': 0,
    'torso': 1,
    'upper extremity': 2,
    'head/neck': 3,
    'palms/soles': 4,
    'oral/genital': 5,
    'unknown': 6,
    'hands/feet': 7
}

In [4]:
# create a new column with the mapping based on dx
dataset['dx'] = dataset['dx'].map(mapping_dx)
dataset['localization'] = dataset['localization'].map(mapping_localization)


In [5]:
# Define input and output shape
input_shape = (224, 224, 3)  # assuming RGB images of size 224x224
age_shape =  (1,)
sex_shape = (1,)
localization_shape = (1,)
num_classes = 1000  # assuming the number of output classes is 1
localization_shape = (1,)


age = np.array(dataset['age'], dtype='float16')
dx = np.array(dataset['dx'], dtype='float16')
images = np.array([img_to_array(img) for img in dataset['image']], dtype='float16')
images = np.repeat(images, 3, axis=3)
localizations = np.array(dataset['localization'], dtype='float16')
sex = dataset['sex'].to_numpy().astype('float16')
sex = sex.reshape(-1,1)
# rescale the images to 0-1
images = images / 255.0

tensor_age = tf.keras.Input(shape=age_shape, name='age', dtype='float16')
tensor_sex = tf.keras.Input(shape=sex_shape, name='sex',dtype='float16')
tensor_localization = tf.keras.Input(shape=localization_shape, name='localization',dtype='float16')
tensor_image = tf.keras.Input(shape=input_shape, name='image',dtype='float16')

base_model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet')(tensor_image)
x = tf.keras.layers.Flatten()(base_model)

# Freeze the base model
base_model.trainable = False
# x = tf.keras.layers.concatenate([x, tensor_age,tensor_sex,tensor_localization])    
# Define fully connected layers
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
# output layer one neuron for each class
x = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)

# Define output layer
predictions = tf.keras.layers.Dense(num_classes)(x)

# Define the model
# model = tf.keras.models.Model(inputs=[tensor_image,tensor_age,tensor_sex,tensor_localization], outputs=predictions)
model = tf.keras.models.Model(inputs=[tensor_image,tensor_age,tensor_sex,tensor_localization], outputs=predictions)

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

2023-02-26 00:29:44.591566: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-26 00:29:44.650386: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-26 00:29:44.650578: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-26 00:29:44.651624: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

In [6]:
train_data = images[:int(len(images)*0.8)]
train_age = age[:int(len(age)*0.8)]
train_sex = sex[:int(len(age)*0.8)]
train_localization = localizations[:int(len(localizations)*0.8)]
train_label = dx[:int(len(dx)*0.8)]

val_data = images[int(len(images)*0.8):int(len(images)*0.9)]
val_age = age[int(len(images)*0.8):int(len(images)*0.9)]
val_sex = sex[int(len(images)*0.8):int(len(images)*0.9)]
val_localization = localizations[int(len(images)*0.8):int(len(images)*0.9)]
val_labels = dx[int(len(images)*0.8):int(len(images)*0.9)]

test_data = images[int(len(images)*0.9):]
test_age = age[int(len(images)*0.9):]
test_sex = sex[int(len(images)*0.9):]
test_localization = localizations[int(len(images)*0.9):]
test_labels = dx[int(len(images)*0.9):]

In [7]:
history = model.fit(
    [train_data,train_age,train_sex,train_localization], train_label,epochs=12, batch_size=32, validation_data=([val_data,val_age,val_sex,val_localization], val_labels)
)

Epoch 1/12


2023-02-26 00:29:50.153376: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8401
2023-02-26 00:29:52.477942: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


19/19 [==============================] - 14s 357ms/step - loss: 0.9148 - accuracy: 0.8419 - val_loss: 0.3875 - val_accuracy: 0.8904
Epoch 2/12
19/19 [==============================] - 4s 203ms/step - loss: 0.3501 - accuracy: 0.8935 - val_loss: 0.3459 - val_accuracy: 0.8904
Epoch 3/12
19/19 [==============================] - 4s 203ms/step - loss: 0.3481 - accuracy: 0.8935 - val_loss: 0.3517 - val_accuracy: 0.8904
Epoch 4/12
19/19 [==============================] - 4s 205ms/step - loss: 0.3520 - accuracy: 0.8935 - val_loss: 0.3495 - val_accuracy: 0.8904
Epoch 5/12
19/19 [==============================] - 4s 205ms/step - loss: 0.3443 - accuracy: 0.8935 - val_loss: 0.3597 - val_accuracy: 0.8904
Epoch 6/12
19/19 [==============================] - 4s 206ms/step - loss: 0.3458 - accuracy: 0.8935 - val_loss: 0.3503 - val_accuracy: 0.8904
Epoch 7/12
19/19 [==============================] - 4s 207ms/step - loss: 0.3416 - accuracy: 0.8935 - val_loss: 0.3459 - val_accuracy: 0.8904
Epoch 8/12
19/19

In [8]:
test_loss, test_acc = model.evaluate([test_data,test_age,test_sex,test_localization], test_labels)

5/5 [==============================] - 0s 52ms/step - loss: 0.3662 - accuracy: 0.8904


In [9]:
print('Test accuracy:', test_acc)

Test accuracy: 0.8904109597206116


In [10]:
tf.keras.Model.save(model, 'test.h5')

/home/mad/mambaforge/envs/MachineLearning/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
